In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Freq_Dataframes") \
        .master('local[*]') \
        .getOrCreate()

#spark = SparkSession.builder.appName("Dataframes") \
#        .getOrCreate()

In [ ]:
from pyspark.sql.functions import split, lower, explode

In [ ]:
df = spark.createDataFrame([('Este es mi texto',)], ['col1'])
df.show()

In [ ]:
df.select( explode( split( lower(df.col1), " ") ).alias("palabras") ).show()